In [6]:
!pip install mysql-connector-python

In [77]:
import mysql.connector as mysql
#connection = sqlite3.connect('backup.db')


try:
    connection = mysql.connect(host="localhost", database="test_db", user="root", password="sveinung", port=3306)
except Exception as e:
    print("ERROR: Failed to connect to db:", e)
cursor = connection.cursor()

In [8]:
cursor.execute("""
SELECT count(id)
FROM User
UNION ALL
SELECT count(id)
FROM Activity
UNION ALL
SELECT count(id)
FROM TrackPoint
""")

cursor.fetchall()

[(182,), (16030,), (9636829,)]

In [9]:
cursor.execute("""
    SELECT 
        AVG(trackpoints_count) AS average_trackpoints, 
        MAX(trackpoints_count) AS maximum_trackpoints, 
        MIN(trackpoints_count) AS minimum_trackpoints
    FROM (
        SELECT 
            User.id, 
            COUNT(TrackPoint.id) AS trackpoints_count
        FROM User
        INNER JOIN Activity ON User.id = Activity.user_id
        INNER JOIN TrackPoint ON Activity.id = TrackPoint.activity_id
        GROUP BY User.id
    ) AS trackpoints_per_user
""")

cursor.fetchall()


[(Decimal('55704.2139'), 1007827, 17)]

In [10]:
cursor.execute("""
    SELECT 
        User.id, 
        COUNT(DISTINCT Activity.id) AS activity_count
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    GROUP BY User.id
    ORDER BY activity_count DESC
    LIMIT 15;
""")

cursor.fetchall()


[('128', 2101),
 ('153', 1791),
 ('025', 715),
 ('163', 704),
 ('062', 691),
 ('144', 563),
 ('041', 398),
 ('085', 364),
 ('004', 346),
 ('140', 345),
 ('167', 320),
 ('068', 278),
 ('017', 265),
 ('003', 261),
 ('014', 232)]

In [11]:
cursor.execute("""
    SELECT DISTINCT User.id
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    WHERE Activity.transportation_mode = 'bus';
""")
cursor.fetchall()

[('010',),
 ('052',),
 ('062',),
 ('073',),
 ('081',),
 ('084',),
 ('085',),
 ('091',),
 ('092',),
 ('112',),
 ('125',),
 ('128',),
 ('175',)]

5. 

In [12]:
cursor.execute("""
    SELECT 
        User.id, 
        COUNT(DISTINCT Activity.transportation_mode) AS mode_count
    FROM User
    INNER JOIN Activity ON User.id = Activity.user_id
    WHERE Activity.transportation_mode != 'NULL'
    GROUP BY User.id
    ORDER BY mode_count DESC
    LIMIT 10;
""")
cursor.fetchall()

[('128', 9),
 ('062', 7),
 ('085', 4),
 ('084', 3),
 ('163', 3),
 ('058', 3),
 ('081', 3),
 ('078', 3),
 ('112', 3),
 ('020', 2)]

In [13]:
cursor.execute("""
    SELECT 
        user_id, 
        transportation_mode, 
        start_date_time, 
        end_date_time, 
        COUNT(*) AS occurrence_count
    FROM Activity
    GROUP BY user_id, transportation_mode, start_date_time, end_date_time
    HAVING COUNT(*) > 1;
""")
cursor.fetchall()

[]

1. a) Not sure if the task asks for us to include the activities that are not classified, we therefore make the assumption that we should not list the 'NULL' activities.

In [14]:
cursor.execute("""
    SELECT 
        COUNT(DISTINCT user_id) 
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time) AND transportation_mode != 'NULL';
""")


cursor.fetchall()

[(9,)]

b) Same assumption

In [15]:
cursor.execute("""
    SELECT 
        user_id, 
        transportation_mode, 
        TIMESTAMPDIFF(MINUTE, start_date_time, end_date_time) AS duration_minutes
    FROM Activity
    WHERE DATE(start_date_time) != DATE(end_date_time) AND transportation_mode != 'NULL';
""")


cursor.fetchall()

[('020', 'bike', 610),
 ('021', 'walk', 237),
 ('058', 'car', 36),
 ('062', 'walk', 83),
 ('085', 'bus', 58),
 ('115', 'car', 88),
 ('115', 'car', 62),
 ('115', 'car', 60),
 ('115', 'car', 65),
 ('115', 'car', 83),
 ('115', 'car', 54),
 ('115', 'car', 68),
 ('115', 'car', 69),
 ('115', 'car', 63),
 ('115', 'car', 61),
 ('115', 'car', 55),
 ('115', 'car', 84),
 ('115', 'car', 77),
 ('115', 'car', 61),
 ('115', 'car', 63),
 ('115', 'car', 58),
 ('115', 'car', 466),
 ('115', 'car', 65),
 ('115', 'car', 57),
 ('115', 'car', 68),
 ('126', 'walk', 7),
 ('128', 'car', 64),
 ('128', 'car', 44),
 ('128', 'bus', 58),
 ('128', 'subway', 30),
 ('128', 'car', 33),
 ('128', 'car', 100),
 ('128', 'airplane', 331),
 ('128', 'subway', 27),
 ('163', 'bike', 11)]

8.

In [16]:
cursor.execute("""
SELECT user.id, date, lat,lon FROM Trackpoint
INNER JOIN Activity
ON Activity.id = Trackpoint.activity_id
INNER JOIN user
on user.id = activity.user_id
ORDER BY date ASC

""")




result = cursor.fetchall()

In [18]:
import haversine as hs



In [19]:
meetups = []
for i in range(len(result)):
    user1 = result[i]
    user1_id = user1[0]
    user1_date = user1[1]
    user1_lat = user1[2]
    user1_lon = user1[3]
    for j in range(i+1,len(result)):
        user2 = result[j]
        user2_id = user2[0]
        if user1_id == user2_id: continue
        user2_date = user2[1]
        seconds_difference = (user2_date - user1_date).total_seconds()
        if seconds_difference > 30: break
        user2_lat = user2[2]
        user2_lon = user2[3]
        distance = hs.haversine((user1_lat,user1_lon),(user2_lat,user2_lon))
        if distance > 0.05: continue
        meetups.append((user1_id,user2_id))
    
        
    
        

    #print((date -date).total_seconds())
    

In [26]:
dic = {}
for tup in meetups:
    id1 = tup[0]
    id2 = tup[1]
    if id1 not in dic: dic[id1] = {id2}
    else: dic[id1].add(id2)
    if id2 not in dic: dic[id2] = {id1}
    else: dic[id2].add(id1)

print("number of users who have meet eachother:", len(dic))




number of users who have meet eachother: 121


9.

In [34]:
cursor.execute("""
SELECT
    u.id AS user_id,
    SUM(CASE WHEN tp1.altitude > tp2.altitude THEN tp1.altitude - tp2.altitude ELSE 0 END) AS total_altitude_gain
FROM
    User AS u
JOIN
    Activity AS a ON u.id = a.user_id
JOIN
    TrackPoint AS tp1 ON a.id = tp1.activity_id
JOIN
    TrackPoint AS tp2 ON a.id = tp2.activity_id AND tp1.id = tp2.id - 1
WHERE
    tp1.altitude != 'NULL' AND tp2.altitude != 'NULL'
GROUP BY
    u.id
ORDER BY
    total_altitude_gain DESC
LIMIT
    15;

""")

cursor.fetchall()

[('153', Decimal('2062756')),
 ('128', Decimal('1885825')),
 ('004', Decimal('1144727')),
 ('003', Decimal('793859')),
 ('163', Decimal('770091')),
 ('085', Decimal('753055')),
 ('062', Decimal('670128')),
 ('041', Decimal('655190')),
 ('144', Decimal('568877')),
 ('030', Decimal('563027')),
 ('039', Decimal('511261')),
 ('025', Decimal('470196')),
 ('084', Decimal('453032')),
 ('167', Decimal('417595')),
 ('000', Decimal('413662'))]

In [76]:
cursor.execute("""


WITH distance AS (
    SELECT
        a.user_id,
        a.transportation_mode,
        SUM(
            6371 * ACOS(
                COS(RADIANS(tp1.lat)) * COS(RADIANS(tp2.lat)) * COS(RADIANS(tp2.lon) - RADIANS(tp1.lon)) +
                SIN(RADIANS(tp1.lat)) * SIN(RADIANS(tp2.lat))
            )
        ) AS total_distance
    FROM
        Activity AS a
    JOIN
        TrackPoint AS tp1 ON a.id = tp1.activity_id
    JOIN drop the files 
        TrackPoint AS tp2 ON a.id = tp2.activity_id AND tp1.id = tp2.id - 1
    WHERE
        a.transportation_mode != 'NULL'
    GROUP BY
        a.user_id, a.transportation_mode
)
SELECT *
FROM distance



""")

cursor.fetchall()

InterfaceError: 2013: Lost connection to MySQL server during query

11.

In [28]:
cursor.execute("""
SELECT 
    user_id, 
    transportation_mode
FROM 
    (SELECT 
        user_id, 
        transportation_mode, 
        RANK() OVER(PARTITION BY user_id ORDER BY COUNT(*) DESC) as rnk
     FROM Activity
     WHERE transportation_mode != 'NULL'
     GROUP BY user_id, transportation_mode) AS Ranked
WHERE 
    rnk = 1
ORDER BY 
    user_id ASC;

""")

cursor.fetchall()

[('010', 'taxi'),
 ('020', 'bike'),
 ('021', 'walk'),
 ('052', 'bus'),
 ('056', 'bike'),
 ('058', 'taxi'),
 ('058', 'car'),
 ('058', 'walk'),
 ('060', 'walk'),
 ('062', 'walk'),
 ('062', 'bus'),
 ('064', 'bike'),
 ('065', 'bike'),
 ('067', 'walk'),
 ('069', 'bike'),
 ('073', 'walk'),
 ('075', 'walk'),
 ('076', 'car'),
 ('078', 'walk'),
 ('080', 'taxi'),
 ('080', 'bike'),
 ('081', 'bike'),
 ('082', 'walk'),
 ('084', 'walk'),
 ('085', 'walk'),
 ('086', 'car'),
 ('087', 'walk'),
 ('089', 'car'),
 ('091', 'bus'),
 ('091', 'walk'),
 ('092', 'bus'),
 ('092', 'walk'),
 ('097', 'bike'),
 ('098', 'taxi'),
 ('101', 'car'),
 ('102', 'bike'),
 ('107', 'walk'),
 ('108', 'walk'),
 ('111', 'taxi'),
 ('112', 'walk'),
 ('115', 'car'),
 ('117', 'walk'),
 ('125', 'bike'),
 ('126', 'bike'),
 ('128', 'car'),
 ('136', 'walk'),
 ('138', 'bike'),
 ('139', 'bike'),
 ('144', 'walk'),
 ('153', 'walk'),
 ('161', 'walk'),
 ('163', 'bike'),
 ('167', 'bike'),
 ('175', 'bus')]

12.